# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [ ]:
def execute_select(query, session):
    """Execute Select Query

    execute select query and output Pandora DataFrame

    Args:
        query (string): SELECT query
        session (session): session
    """

    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    rowslist = list(rows)
    if (len(rowslist) == 0):
        print ('no data found')
        return

    # set DataFrame
    df = pd.DataFrame(rowslist)
    print (df)

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Create Pandas DataFrame

In [ ]:
file = 'event_datafile_new.csv'

df = pd.read_csv(file)
df.head()


### Query 1

#### Create table for Query 1 

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS music_app_history_1 (
    artist text, 
    item_in_session int, 
    song_length float,
    session_id int,
    song_title text, 
    PRIMARY KEY (session_id, item_in_session)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)


#### Insert Data fro music_app_history_1

In [ ]:
query = """
 INSERT INTO music_app_history_1 (artist, item_in_session, song_length, session_id, song_title) 
 VALUES (%s, %s, %s, %s, %s)
"""

history1_df = df[['artist', 'itemInSession', 'length', 'sessionId','song']]

for i, row in history1_df.iterrows():
    session.execute(query, list(row))


#### execute Query 1.

In [ ]:
query = "select * from music_app_history_1 WHERE session_id = 338 and item_in_Session = 4"

try:
    execute_select(query, session)
except Exception as e:
    print(e)

### Query 2.
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Create musit_app_history_2 table

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS music_app_history_2 (
    artist text,
    item_in_session int,
    session_id int,
    song_title text,
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id), item_in_session)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)


#### Insert Data into music_app_history_2

In [ ]:
query = """
INSERT INTO music_app_history_2 (artist, item_in_session, session_id, song_title, user_id, first_name, last_name) 
 VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

history2_df = df[['artist', 'itemInSession', 'sessionId', 'song','userId', 'firstName', 'lastName']]

for i, row in history2_df.iterrows():
    session.execute(query, list(row))


#### execute Query 2.

In [ ]:
query = "select * from music_app_history_2 WHERE user_id = 10 and session_id = 182"

execute_select(query, session)


### Query 3.
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create music_app_history_3 Table

In [ ]:
query = """
CREATE TABLE IF NOT EXISTS music_app_history_3 (
    song_title text, 
    user_id int,
    first_name text,
    last_name text,
    PRIMARY KEY (song_title, user_id)
)
"""

try:
    session.execute("DROP TABLE IF EXISTS music_app_history_3")
    session.execute(query)
except Exception as e:
    print(e)


#### Insert Data into music_app_history_3

In [ ]:
query = """
INSERT INTO music_app_history_3 (song_title, user_id, first_name, last_name) 
 VALUES (%s, %s, %s, %s)
"""

history3_df = df[['song', 'userId', 'firstName', 'lastName']]

for i, row in history3_df.iterrows():
        session.execute(query, list(row))

#### execute Query 3

In [ ]:
query = "select * from music_app_history_3 WHERE song_title = 'All Hands Against His Own'"

execute_select(query, session)

### Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute("DROP TABLE IF EXISTS music_app_history_1")
    session.execute("DROP TABLE IF EXISTS music_app_history_2")
    session.execute("DROP TABLE IF EXISTS music_app_history_3")
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()